__Inference set up - FMD_to_Hagen_Transfer__

In [1]:
from fastai.vision.all import *
import os
import pathlib
import cv2
import matplotlib.pyplot as plt
import fastai; fastai.__version__

'2.7.15'

In [2]:
torch.cuda.set_device(0)

In [3]:
torch.cuda.current_device(), torch.cuda.get_device_name(0)

(0, 'NVIDIA GeForce RTX 4060 Ti')

In [4]:
device=torch.device("cuda")

In [5]:
path = pathlib.Path(os.getcwd())

In [6]:
path_GT = path/'Daten/Hagen_GT'
path_noisy = path/'Daten/Hagen_noisy'

In [7]:
%run Dataloader_Hagen.ipynb
%run Perception_loss.ipynb

/home/david/mambaforge/lib/python3.10/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/david/mambaforge/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
dls_den, dblock = get_dls(38)  # required to generate learner 

In [9]:
loss_func = FeatureLoss(vgg_m, blocks[2:5],[1], [1,1,1], [1,1,1]) # dummy

In [10]:
# define learner w/o learning parameters (e.g. metrics, cbs)
def create_gen_learner():
    return unet_learner(dls_den, bbone, loss_func=loss_func, blur=True, norm_type=NormType.Weight, 
                        self_attention=True, y_range=y_range
                        )

In [11]:
# create learner
learn_den = create_gen_learner()
# load the Basemodel into that leaner,
learn_den.load('FMD_to_Hagen').to_fp16()
# learn_den.load('FMD_to_Hagen_all').to_fp16() # this model was pretrained using FMD data and retrained using all Hagen data

/home/david/mambaforge/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


__Set up data and perform inference__

In [12]:
path_test = path/'Daten/Hagen_testmix/noisy_tiles'
path_preds = path/'Daten/Hagen_testmix/Transfer_pred_tiles'
# path_preds = path/'Daten/Hagen_testmix/Transfer_all_pred_tiles' # define new folder for new model, e.g. model using all Hagen data 
path_preds.mkdir(exist_ok=True)

In [13]:
# set path
test_files = get_image_files(path_test, recurse=True)
# check path
test_files[0], len(test_files)

(Path('/home/david/mnt_anarchyNAS/lohrd/Github/AIMD/Daten/Hagen_testmix/noisy_tiles/mito-20x-noise1_frame_90_tile_64_.png'),
 16408)

In [14]:
def save_preds(dl, learn):
  "Save away predictions"
  names = dl.dataset.items  
    
  inp, preds, _,decoded = learn.get_preds(dl=dl, with_input=True, with_decoded=True)

  for i,pred in enumerate(preds):      
      arr = pred.clamp_(min=0,max=1).permute(1,2,0).numpy()
      arr = arr[:,:,0]*(2**16-1)
      cv2.imwrite(str(path_preds/names[i].name),arr.astype(np.uint16))
  return inp, preds, decoded       # return outputs for analysis

In [15]:
"""
test_dl=learn_den.dls.test_dl(test_files)
inp, preds, dec = save_preds(test_dl, learn_den)
"""

'\ntest_dl=learn_den.dls.test_dl(test_files)\ninp, preds, dec = save_preds(test_dl, learn_den)\n'

In [16]:
# use this loop with a chunksize, if your RAM cant contain all predictions
chunk_size = 5000
for k in range(0,len(test_files),chunk_size):
    chunk = test_files[k:k+chunk_size]
    test_dl=learn_den.dls.test_dl(chunk)
    inp, preds, dec = save_preds(test_dl, learn_den)

/home/david/mambaforge/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at /opt/conda/conda-bld/pytorch_1716905971132/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,
